In [1]:
import pandas as pd
import numpy as np
import json
import requests
from warcio import ArchiveIterator

In [2]:
files = pd.read_csv('Data/sept_files.csv', header=None)
files = list(files[0])
len(files)

79600

In [3]:
all_sides = pd.read_csv('Data/allsides.csv')
all_sides.head()

,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,21052,1.680530,Agrees,https://www.allsides.com/news-source/abc-news-...,left-center,12527,ABC News (Online),33579
1,80,2.857143,Strongly Agrees,https://www.allsides.com/news-source/above-law...,left-center,28,Above The Law,108
2,223,1.581560,Agrees,https://www.allsides.com/news-source/abridge-n...,allsides,141,Abridge News,364
3,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
4,4217,0.631382,Disagrees,https://www.allsides.com/news-source/al-jazeer...,left-center,6679,Al Jazeera,10896


In [4]:
def find_url(url):
    return url.replace('https://www.allsides.com/news-source/', '').replace('media-bias', '').replace('-', '')

In [5]:
all_sides['url_match'] = all_sides['allsides_page'].apply(find_url)
all_sides.head()

,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes,url_match
0,21052,1.680530,Agrees,https://www.allsides.com/news-source/abc-news-...,left-center,12527,ABC News (Online),33579,abcnews
1,80,2.857143,Strongly Agrees,https://www.allsides.com/news-source/above-law...,left-center,28,Above The Law,108,abovelaw
2,223,1.581560,Agrees,https://www.allsides.com/news-source/abridge-n...,allsides,141,Abridge News,364,abridgenews
3,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369,aj
4,4217,0.631382,Disagrees,https://www.allsides.com/news-source/al-jazeer...,left-center,6679,Al Jazeera,10896,aljazeera


In [6]:
sources = all_sides['url_match'].unique()

In [7]:

text = []
urls = []
t = 0
for file in files[201:300]:
    warc_url = 'https://commoncrawl.s3.amazonaws.com/{}'.format(file)
    wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
    r = requests.get(wet_url, stream=True)
    records = ArchiveIterator(r.raw)
    try:
        i=0
        while i < 100000:
            record = next(records)
            for r1, r2 in record.rec_headers.headers:
                if r1 == 'WARC-Identified-Content-Language' and 'eng' == r2:
                    a = record.content_stream().read()
                    for r1_1, r2_1 in record.rec_headers.headers:
                        if r1_1 == 'WARC-Target-URI':
                            for source in sources:
                                if source in r2_1:
                                    urls.append(source)
                                    text.append(a.decode('utf-8'))
            i += 1
    except:
        t += 1
        print(t)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
text_df = pd.DataFrame({'text': text,
                       'url_match': urls})
full_data = pd.merge(text_df, all_sides, how='left', on='url_match')
print(full_data.shape)
full_data.head()

(101796, 10)


,text,url_match,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,Sign In | Sign Up Sign In | Sign Up\nBikes\nNe...,aj,263,2.481132,Strongly Agrees,https://www.allsides.com/news-source/aj-media-...,left,106,AJ+,369
1,accrue.info - For Sale!\nThis domain is for sa...,mit,4,1.333333,Somewhat Agrees,https://www.allsides.com/news-source/mit-media...,left,3,Mitú,7
2,Kontrag gesetz online dating :: Friesland zorg...,bet,131,0.803681,Somewhat Disagrees,https://www.allsides.com/news-source/bet-media...,left-center,163,BET,294
3,Our Services\nHome Careers About Us\n(714) 282...,hill,10563,0.794509,Somewhat Disagrees,https://www.allsides.com/news-source/hill-medi...,center,13295,The Hill,23858
4,"Caimes, Author at\nHome\nAnthill Magazine\nFun...",hill,10563,0.794509,Somewhat Disagrees,https://www.allsides.com/news-source/hill-medi...,center,13295,The Hill,23858


In [9]:
full_data.to_csv('cleaned_data_3.csv', index=False)